In [1]:
%run src_runs/autorun/daily_update.py

Use device name: NVIDIA GeForce RTX 4090
update data: ********************
Unpack Update Files
Sat Nov 16 13:39:25 2024 : download since!
Connection and Factor preparation finished!
Sat Nov 16 13:39:25 2024 : All Updates Done! Cost 0.50 Secs
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
process other min bars:
--------------------------------------------------------------------------------
process other min bars:
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
prepare predict data: ********************
predict is True , Data Processing start!
6 datas :['y', 'day', '30m', 'style', 'indus', 'week']
y is up to 20241116 already!
day is up to 20241116 already!
30m is up to 20241116 alr

100%|██████████| 333/333 [00:00<00:00, 3737.91it/s]


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
update_factors: ********************
Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
sending email success


In [1]:
from src.api import DataAPI , ModelAPI

#print('update_data: ' + '*' * 20)
#DataAPI.update()

#ModelAPI.update()


Use device name: NVIDIA GeForce RTX 4090


In [8]:
from src.data.tushare.download.info import Calendar , pro
renamer = {'cal_date' : 'calendar' ,'is_open'  : 'trade'}
fields = None
df = pro.trade_cal(exchange='SSE').rename(columns=renamer)
#df = df.sort_values('calendar').reset_index(drop = True)
df

""


In [ ]:
from src.api import ModelAPI
model = ModelAPI.Testor('ts_mixer')
model.try_forward()
model.try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
